# Elements of Convex Optimization 2024 - Project

### Solution author: <name, index>

This notebook uses helper classes and functions defined in [eco_project_helpers](eco_project_helpers) file, but there is no need to look at it.

This project requires **numpy**, **matplotlib**, **seaborn** and **[autograd](https://github.com/HIPS/autograd)** libraries.

In [ ]:
!pip install numpy matplotlib seaborn autograd

## Task

The task is to implement an optimization algorithm that can deal with non-convex, multi-modal functions that are differentiable and smooth over the entire domain. Functions can have a varying number of arguments (more than 2). The quality of the solution is measured by the success rate of finding the global minimum of the function in a given budget of function (or gradient/Hessian) evaluations. The final grades will be calculated relative to the best-submitted solution. The best solution will be awarded with bonus points (+10 bonus percent point to the final grade).

## Benchmark functions

Below, you can find a few example benchmark functions. 
Please be aware that the final evaluation functions, which may have more than two arguments, could be more complex than the examples provided. These examples are a good starting point for testing your solution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from eco_project_helpers import BENCHMARK_FUNCTIONS, BenchmarkFunction


def plot_function_2d(f: BenchmarkFunction):
    if f.N == 2: # Plot only if function is 2D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x = np.linspace(f.bounds[0][0], f.bounds[0][1], 100)
        y = np.linspace(f.bounds[1][0], f.bounds[1][1], 100)
        X, Y = np.meshgrid(x, y)
        Z = np.array([[f.func(np.array([x, y])) for x, y in zip(x, y)] for x, y in zip(X, Y)])
        ax.plot_surface(X, Y, Z, cmap=sns.color_palette("flare", as_cmap=True))
        plt.show()


def check_grad_and_hessian(f: BenchmarkFunction):
    x_1 = np.linspace(f.bounds[0][0], f.bounds[0][1], 100)
    x_2 = np.linspace(f.bounds[1][0], f.bounds[1][1], 100)
    for x_1_i in x_1:
        for x_2_i in x_2:
            x = np.array([x_1_i, x_2_i])
            grad = f.grad(x)
            H = f.hess(x)
            if np.isnan(grad).any() or np.isnan(H).any():
                print(f"NaN gradient or Hessian at {x}")
                return


for func_class in BENCHMARK_FUNCTIONS:
    f = func_class(budget=None)
    print(f"Function: {f.__class__.__name__}")
    print(f"Dimensions: {f.N}")
    print(f"Bounds: {f.bounds}")
    plot_function_2d(f)
    check_grad_and_hessian(f) # This may take some time

## Example implementations of naive solutions

In [ ]:
def random_sampling(f: BenchmarkFunction):
    best_val = np.inf
    best_x = None
    while f.budget_left:
        x = np.random.uniform(f.xmin, f.xmax)
        val = f.func(x)

        if val < best_val:
            best_val = val
            best_x = x

    return best_x


def naive_gradient_descent(f: BenchmarkFunction):
    start_learnign_rate = 0.1
    x = np.random.uniform(f.xmin, f.xmax) # Random initial point
    budget = f.budget_left
    for i in range(budget):
        lr = start_learnign_rate * (budget - i) / budget # Linearly decrease learning rate
        grad = f.grad(x)
        new_x = x - lr * grad

        # Only move to new point if it is within bounds
        if np.all(new_x >= f.xmin) and np.all(new_x <= f.xmax):
            x = new_x

    return x

## Evaluation

Your solution will be evaluated similarly to the code below. The evaluation will be done on a set of benchmark functions that are not presented here. The x returned by the function will be compared to the global minimum of the function. The returned x will be considered the global minimum if the value of the function in x is less than 1e-5, then the value of the global minimum.

In [ ]:
from typing import Callable, List

def eval_optimization_method(
        opt_f: Callable[[BenchmarkFunction], np.ndarray], 
        benchmarks: List[BenchmarkFunction], 
        budget: int = 1000, 
        repetitions: int = 10               
    ):
    for func_class in benchmarks:
        success_rate = 0
        for _ in range(repetitions):
            f = func_class(budget=budget)
            x = opt_f(f)
            if f.success(x):
                success_rate += 1
        success_rate /= repetitions
        print(f"Success rate of {opt_f.__name__}({f.__class__.__name__}(budget={budget})) = {success_rate}")

eval_optimization_method(random_sampling, BENCHMARK_FUNCTIONS, budget=1000)
eval_optimization_method(naive_gradient_descent, BENCHMARK_FUNCTIONS, budget=1000)

## Your solution

Implement your solution below. DO NOT change the name of the function, and DO NOT add positional arguments to it (keyword arguments with default values are ok), as it will be graded and evaluated automatically.

In [ ]:
def my_global_optimization_algorithm(f: BenchmarkFunction):
    # All available tools:
    dim = f.N # Number of input dimensions (function arguments)
    x = np.random.uniform(f.xmin, f.xmax)
    budget = f.budget_left # Remaining budget

    val = f.func(x) # Evaluate function at x, reduce budget by 1
    grad = f.grad(x) # Evaluate gradient at x, reduce budget by 1
    H = f.hess(x) # Evaluate Hessian at x, reduce budget by 1

    # You implement this here

    return x

Tips: 
- Check how your function performs on the benchmark functions with different budgets (e.g., 100, 500, 1000, 2000). 
- Use a large number of repetitions to get a better estimate of the success rate of your solution.

In [ ]:
eval_optimization_method(my_global_optimization_algorithm, BENCHMARK_FUNCTIONS, budget=1000, repetitions=100)